In [1]:
import pickle
import pandas as pd
import numpy as np
from ml_assemblr.main_components.data_pod import DataPod
from ml_assemblr.main_components.transformer import Serializer
from ml_assemblr.utils.df.columns_utils import get_categorical_column_names
from home_credit_helper.config import cfg, cfg_small_dataset
from home_credit_helper.constant import *

In [2]:
from ml_assemblr.transfromer.unfit.value_replacer import ValueReplacer
from ml_assemblr.transfromer.unfit.bicolumns_operator import BicolumnsOperator
from ml_assemblr.transfromer.encoder.one_hot_encoder import OneHotEncoder
from ml_assemblr.transfromer.encoder.label_encoder import LabelEncoder
from ml_assemblr.transfromer.unfit.aggregator import Aggregator

In [3]:
from home_credit_helper.utils.filter_functions import (
    bureaus_credit_active_filter_function,
    bureaus_credit_closed_filter_function,
    prev_apps_approved_filter_function,
    prev_apps_refused_filter_function,
)
from home_credit_helper.utils.bicolumns_operation import get_dbd, get_dpd
from home_credit_helper.cleaning_pipeline.factory import get_cleaning_pipeline

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
cfg = cfg_small_dataset
del cfg_small_dataset

In [6]:
with open(cfg.research_cache_path / "01_dp.pkl", "rb") as f:
    dp: DataPod = pickle.load(f)

# 0. Clean data

In [7]:
# cleaning_pipeline = get_cleaning_pipeline(set(dp.dfs.keys()))
# dp = dp.fit_transform(cleaning_pipeline)

# 1. Applications

In [8]:
dp.peek_main_df()

20000 rows × 123 columns


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,train
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [9]:
replacing_mappers = {"DAYS_EMPLOYED": {365243: np.nan}, "CODE_GENDER": {'XNA': np.nan}}
value_replacer = ValueReplacer(replacing_mappers=replacing_mappers)
dp = dp.fit_transform(value_replacer)

In [10]:
binary_col_label_encoder = LabelEncoder(col_names=['FLAG_OWN_CAR', 'FLAG_OWN_REALTY'])
dp = dp.fit_transform(binary_col_label_encoder)

In [11]:
bicolumns_configs = [
    ("DAYS_EMPLOYED_PERC", "DAYS_EMPLOYED", "/", "DAYS_BIRTH"),
    ("INCOME_CREDIT_PERC", "AMT_INCOME_TOTAL", "/", "AMT_CREDIT"),
    ("INCOME_PER_PERSON", "AMT_INCOME_TOTAL", "/", "CNT_FAM_MEMBERS"),
    ("ANNUITY_INCOME_PERC", "AMT_ANNUITY", "/", "AMT_INCOME_TOTAL"),
    ("PAYMENT_RATE", "AMT_ANNUITY", "/", "AMT_CREDIT"),
]
bicolumns_pipeline = []
for target_col_name, first_col_name, operation, second_col_name in bicolumns_configs:
    bicolumns_pipeline.append(
        BicolumnsOperator(
            target_col_name=target_col_name,
            first_col_name=first_col_name,
            operation=operation,
            second_col_name=second_col_name,
        )
    )
bicolumns_pipeline = Serializer(transformers=bicolumns_pipeline)

dp = dp.fit_transform(bicolumns_pipeline)

In [12]:
df = dp.main_df
categorical_column_names = get_categorical_column_names(df, exclude={SPLIT})
one_hot_encoder = OneHotEncoder(target_df_name=APPLICATIONS, col_names=categorical_column_names)
dp = dp.fit_transform(one_hot_encoder)

In [13]:
dp.peek_main_df()

20000 rows × 168 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN
0,100002,1.0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.

# 2. Bureaus / Bureau_Balances

In [14]:
dp.peek_df(BUREAU_BALANCES)

2405172 rows × 3 columns


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
2289,5718724,0,0
2290,5718724,-1,0
2291,5718724,-2,0
2292,5718724,-3,0
2293,5718725,0,C


In [15]:
bb_df = dp.dfs[BUREAU_BALANCES]
bb_cat_col_names = [col for col in bb_df.columns if bb_df[col].dtype == 'object']
one_hot_encoder = OneHotEncoder(target_df_name=BUREAU_BALANCES, col_names=bb_cat_col_names, fit_on_split=None)
dp = dp.fit_transform(one_hot_encoder)

In [16]:
dp.peek_df(BUREAU_BALANCES)

4560290 rows × 6 columns


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_C,STATUS_X,STATUS_INFREQUENT_SKLEARN
2289,5718724.0,0.0,0.0,0.0,1.0,0.0
2290,5718724.0,-1.0,0.0,0.0,1.0,0.0
2291,5718724.0,-2.0,0.0,0.0,1.0,0.0
2292,5718724.0,-3.0,0.0,0.0,1.0,0.0
2293,5718725.0,0.0,0.0,0.0,1.0,0.0


In [17]:
dp.peek_df(BUREAUS)

98561 rows × 17 columns


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
864,101060,5715559,Closed,currency 1,-1087,0,-874.0,-902.0,NaN,0,74242.80,0.00,NaN,0.0,Consumer credit,-711,NaN
865,101060,5715560,Active,currency 1,-709,0,783.0,NaN,NaN,0,395940.69,270998.82,NaN,0.0,Consumer credit,-7,NaN
866,101060,5715561,Active,currency 1,-886,0,479.0,NaN,NaN,0,270000.00,137875.50,0.0,0.0,Credit card,-26,NaN
867,101060,5715564,Closed,currency 1,-2632,0,-2478.0,-2541.0,0.0,0,76392.00,0.00,0.0,0.0,Consumer credit,-2539,NaN
924,104261,5715630,Active,currency 1,-587,0,872.0,NaN,NaN,0,450000.00,317925.00,0.0,0.0,Consumer credit,-6,14301.0


In [18]:
bb_agg_funcs = {"MONTHS_BALANCE": ["min", "max", "size"]}
bb_one_hot_col_names = dp.footprints.transformers[-1].target_col_names
for col in bb_one_hot_col_names:
    bb_agg_funcs[col] = ['mean']

bb_aggregator = Aggregator(
    target_df_name=BUREAUS,
    source_df_name=BUREAU_BALANCES,
    groupby_col_names=BUREAU_KEY,
    agg_functions=bb_agg_funcs,
)

dp = dp.fit_transform(bb_aggregator)

In [19]:
dp.peek_df(BUREAUS)

98561 rows × 24 columns


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_INFREQUENT_SKLEARN_MEAN
0,101060,5715559,Closed,currency 1,-1087,0,-874.0,-902.0,NaN,0,74242.80,0.00,NaN,0.0,Consumer credit,-711,NaN,-35.0,-19.0,17.0,0.000000,1.000000,0.000000,0.000000
1,101060,5715560,Active,currency 1,-709,0,783.0,NaN,NaN,0,395940.69,270998.82,NaN,0.0,Consumer credit,-7,NaN,-23.0,0.0,24.0,0.000000,1.000000,0.000000,0.000000
2,101060,5715561,Active,currency 1,-886,0,479.0,NaN,NaN,0,270000.00,137875.50,0.0,0.0,Credit card,-26,NaN,-29.0,0.0,30.0,0.200000,0.766667,0.000000,0.033333
3,101060,5715564,Closed,currency 1,-2632,0,-2478.0,-2541.0,0.0,0,76392.00,0.00,0.0,0.0,Consumer credit,-2539,NaN,-86.0,-57.0,30.0,0.833333,0.000000,0.133333,0.033333
4,104261,5715630,Active,currency 1,-587,0,872.0,NaN,NaN,0,450000.00,317925.00,0.0,0.0,Consumer credit,-6,14301.0,-19.0,0.0,20.0,NaN,NaN,NaN,NaN


In [20]:
df = dp.dfs[BUREAUS]
bureaus_cat_col_names = get_categorical_column_names(df)
one_hot_encoder = OneHotEncoder(target_df_name=BUREAUS, col_names=bureaus_cat_col_names, fit_on_split=None)
dp = dp.fit_transform(one_hot_encoder)

In [21]:
dp.peek_df(BUREAUS)

98561 rows × 29 columns


,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_INFREQUENT_SKLEARN_MEAN,CREDIT_ACTIVE_ACTIVE,CREDIT_ACTIVE_CLOSED,CREDIT_ACTIVE_INFREQUENT_SKLEARN,CREDIT_CURRENCY_CURRENCY_1,CREDIT_CURRENCY_INFREQUENT_SKLEARN,CREDIT_TYPE_CONSUMER_CREDIT,CREDIT_TYPE_CREDIT_CARD,CREDIT_TYPE_INFREQUENT_SKLEARN
0,101060,5715559,-1087,0,-874.0,-902.0,NaN,0,74242.80,0.00,NaN,0.0,-711,NaN,-35.0,-19.0,17.0,0.000000,1.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,101060,5715560,-709,0,783.0,NaN,NaN,0,395940.69,270998.82,NaN,0.0,-7,NaN,-23.0,0.0,24.0,0.000000,1.000000,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,101060,5715561,-886,0,479.0,NaN,NaN,0,270000.00,137875.50,0.0,0.0,-26,NaN,-29.0,0.0,30.0,0.200000,0.766667,0.000000,0.033333,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,101060,5715564,-2632,0,-2478.0,-2541.0,0.0,0,76392.00,0.00,0.0,0.0,-2539,NaN,-86.0,-57.0,30.0,0.833333,0.000000,0.133333,0.033333,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,104261,5715630,-587,0,872.0,NaN,NaN,0,450000.00,317925.00,0.0,0.0,-6,14301.0,-19.0,0.0,20.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [22]:
dp.peek_main_df()

20000 rows × 168 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN
0,100002,1.0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.

In [23]:
bureau_agg_funcs = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }

for col in dp.footprints.transformers[-1].target_col_names:
    bureau_agg_funcs[col] = ["mean"]

for col in dp.footprints.transformers[-2].target_col_names:
    if col[-4:] == "MEAN":
        bureau_agg_funcs[col] = ["mean"]

bureau_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=BUREAUS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=bureau_agg_funcs,
    target_col_prefix=BUREAUS,
)

dp = dp.fit_transform(bureau_aggregator)

In [24]:
dp.peek_df(APPLICATIONS)

20000 rows × 207 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [25]:
bureau_active_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=BUREAUS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=bureau_agg_funcs,
    filter_function=bureaus_credit_active_filter_function,
    target_col_prefix=BUREAUS + "_ACTIVE",
)

dp = dp.fit_transform(bureau_active_aggregator)

In [26]:
dp.peek_main_df()

20000 rows × 246 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [27]:
bureau_closed_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=BUREAUS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=bureau_agg_funcs,
    filter_function=bureaus_credit_closed_filter_function,
    target_col_prefix=BUREAUS + "_CLOSED",
)

dp = dp.fit_transform(bureau_closed_aggregator)

In [28]:
dp.peek_main_df()

20000 rows × 285 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

# 3. Previous Application

In [29]:
dp.peek_df(PREV_APPS)

98178 rows × 37 columns


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
17,1285768,142748,Revolving loans,9000.000,180000.0,180000.0,NaN,180000.0,FRIDAY,13,Y,1,NaN,NaN,NaN,XAP,Approved,-336,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,x-sell,AP+ (Cash loan),6,XNA,0.0,XNA,Card X-Sell,-277.0,-257.0,365243.0,365243.0,365243.0,0.0
38,1692577,125157,Consumer loans,9946.485,99319.5,109809.0,0.0,99319.5,SATURDAY,9,Y,1,0.0,NaN,NaN,XAP,Approved,-392,XNA,XAP,Unaccompanied,Repeater,Audio/Video,POS,XNA,Regional / Local,136,Consumer electronics,12.0,low_action,POS household without interest,365243.0,-362.0,-32.0,-242.0,-237.0,1.0
51,2664403,163660,Cash loans,9804.285,67500.0,82611.0,NaN,67500.0,SATURDAY,10,Y,1,NaN,NaN,NaN,Car repairs,Approved,-766,Cash through the bank,XAP,NaN,New,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,12.0,high,Cash Street: high,365243.0,-736.0,-406.0,-466.0,-464.0,1.0
114,1168659,101529,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Refused,-115,XNA,HC,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df = dp.dfs[PREV_APPS]
prev_apps_cat_cols = get_categorical_column_names(df)
one_hot_encoder = OneHotEncoder(target_df_name=PREV_APPS, col_names=prev_apps_cat_cols, fit_on_split=None)
dp = dp.fit_transform(one_hot_encoder)

In [31]:
replacing_mappers = {
    "DAYS_FIRST_DRAWING": {365243: np.nan},
    "DAYS_FIRST_DUE": {365243: np.nan},
    "DAYS_LAST_DUE_1ST_VERSION": {365243: np.nan},
    "DAYS_LAST_DUE": {365243: np.nan},
    "DAYS_TERMINATION": {365243: np.nan},
    
}
prev_apps_value_replacer = ValueReplacer(target_df_name=PREV_APPS, replacing_mappers=replacing_mappers)
dp = dp.fit_transform(prev_apps_value_replacer)

In [32]:
divider = BicolumnsOperator(
    target_df_name=PREV_APPS,
    target_col_name="APP_CREDIT_PERC",
    first_col_name="AMT_APPLICATION",
    operation="/",
    second_col_name="AMT_CREDIT",
)
dp = dp.fit_transform(divider)

In [33]:
dp.peek_df(PREV_APPS)

190735 rows × 93 columns


,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_CONSUMER_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,NAME_CONTRACT_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_Y,FLAG_LAST_APPL_PER_CONTRACT_INFREQUENT_SKLEARN,NAME_CASH_LOAN_PURPOSE_XAP,NAME_CASH_LOAN_PURPOSE_XNA,NAME_CASH_LOAN_PURPOSE_INFREQUENT_SKLEARN,NAME_CONTRACT_STATUS_APPROVED,NAME_CONTRACT_STATUS_CANCELED,NAME_CONTRACT_STATUS_REFUSED,NAME_CONTRACT_STATUS_INFREQUENT_SKLEARN,NAME_PAYMENT_TYPE_CASH_THROUGH_THE_BANK,NAME_PAYMENT_TYPE_XNA,NAME_PAYMENT_TYPE_INFREQUENT_SKLEARN,CODE_REJECT_REASON_HC,CODE_REJECT_REASON_XAP,CODE_REJECT_REASON_INFREQUENT_SKLEARN,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_NAN,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_CLIENT_TYPE_NEW,NAME_CLIENT_TYPE_REFRESHED,NAME_CLIENT_TYPE_REPEATER,NAME_CLIENT_TYPE_INFREQUENT_SKLEARN,NAME_GOODS_CATEGORY_AUDIO_VIDEO,NAME_GOODS_CATEGORY_COMPUTERS,NAME_GOODS_CATEGORY_CONSUMER_ELECTRONICS,NAME_GOODS_CATEGORY_MOBILE,NAME_GOODS_CATEGORY_XNA,NAME_GOODS_CATEGORY_INFREQUENT_SKLEARN,NAME_PORTFOLIO_CARDS,NAME_PORTFOLIO_CASH,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PORTFOLIO_INFREQUENT_SKLEARN,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_WALK_IN,NAME_PRODUCT_TYPE_X_SELL,CHANNEL_TYPE_COUNTRY_WIDE,CHANNEL_TYPE_CREDIT_AND_CASH_OFFICES,CHANNEL_TYPE_REGIONAL_LOCAL,CHANNEL_TYPE_STONE,CHANNEL_TYPE_INFREQUENT_SKLEARN,NAME_SELLER_INDUSTRY_CONNECTIVITY,NAME_SELLER_INDUSTRY_CONSUMER_ELECTRONICS,NAME_SELLER_INDUSTRY_XNA,NAME_SELLER_INDUSTRY_INFREQUENT_SKLEARN,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_HIGH,NAME_YIELD_GROUP_LOW_ACTION,NAME_YIELD_GROUP_LOW_NORMAL,NAME_YIELD_GROUP_MIDDLE,PRODUCT_COMBINATION_CARD_STREET,PRODUCT_COMBINATION_CARD_X_SELL,PRODUCT_COMBINATION_CASH,PRODUCT_COMBINATION_CASH_X_SELL_LOW,PRODUCT_COMBINATION_CASH_X_SELL_MIDDLE,PRODUCT_COMBINATION_POS_HOUSEHOLD_WITH_INTEREST,PRODUCT_COMBINATION_POS_INDUSTRY_WITH_INTEREST,PRODUCT_COMBINATION_POS_MOBILE_WITH_INTEREST,PRODUCT_COMBINATION_INFREQUENT_SKLEARN,APP_CREDIT_PERC
1,2802425.0,108129.0,25188.615,607500.0,679671.0,NaN,607500.0,11.0,1.0,NaN,NaN,NaN,-164.0,-1.0,36.0,NaN,-134.0,916.0,NaN,NaN,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.129001e+11
17,1285768.0,142748.0,9000.000,180000.0,180000.0,NaN,180000.0,13.0,1.0,NaN,NaN,NaN,-336.0,6.0,0.0,-277.0,-257.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.240000e+10
38,1692577.0,125157.0,9946.485,99319.5,109809.0,0.0,99319.5,9.0,1.0,0.0,NaN,NaN,-392.0,136.0,12.0,NaN,-362.0,-32.0,-242.0,-237.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.090617e+10
51,2664403.0,163660.0,9804.285,67500.0,82611.0,NaN,67500.0,10.0,1.0,NaN,NaN,NaN,-766.0,-1.0,12.0,NaN,-736.0,-406.0,-466.0,-

In [34]:
dp.peek_main_df()

20000 rows × 285 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [35]:
prev_apps_agg_funcs = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

for cat_col in dp.footprints.transformers[-3].target_col_names:
    prev_apps_agg_funcs[cat_col] = ["mean"]
    
prev_apps_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=PREV_APPS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=prev_apps_agg_funcs,
    target_col_prefix="PREV_APPS",
)

dp = dp.fit_transform(prev_apps_aggregator)

In [36]:
prev_apps_approved_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=PREV_APPS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=prev_apps_agg_funcs,
    filter_function=prev_apps_approved_filter_function,
    target_col_prefix = "PREV_APPS_APPROVED"
)

dp = dp.fit_transform(prev_apps_approved_aggregator)

In [37]:
prev_apps_approved_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=PREV_APPS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=prev_apps_agg_funcs,
    filter_function=prev_apps_refused_filter_function,
    target_col_prefix = "PREV_APPS_REFUSED"
)

dp = dp.fit_transform(prev_apps_approved_aggregator)

In [38]:
dp.peek_main_df()

20000 rows × 588 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

# 4. POS Cash Balance

In [39]:
dp.peek_df(POS_CASH_BALANCES)

573755 rows × 8 columns


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
31,1463929,108990,-41,12.0,12.0,Active,0,0
35,2701303,109846,-33,18.0,12.0,Active,0,0
116,2105048,144387,-43,24.0,10.0,Active,0,0
149,2415063,171541,-46,24.0,1.0,Active,0,0
175,2820669,165868,-47,14.0,4.0,Active,0,0


In [40]:
df = dp.dfs[POS_CASH_BALANCES]
pos_cash_cat_cols = get_categorical_column_names(df)
one_hot_encoder = OneHotEncoder(target_df_name=POS_CASH_BALANCES, col_names=pos_cash_cat_cols, fit_on_split=None)
dp = dp.fit_transform(one_hot_encoder)

In [41]:
dp.peek_df(POS_CASH_BALANCES)

1114111 rows × 10 columns


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_ACTIVE,NAME_CONTRACT_STATUS_COMPLETED,NAME_CONTRACT_STATUS_INFREQUENT_SKLEARN
31,1463929.0,108990.0,-41.0,12.0,12.0,0.0,0.0,1.0,0.0,0.0
35,2701303.0,109846.0,-33.0,18.0,12.0,0.0,0.0,1.0,0.0,0.0
116,2105048.0,144387.0,-43.0,24.0,10.0,0.0,0.0,1.0,0.0,0.0
149,2415063.0,171541.0,-46.0,24.0,1.0,0.0,0.0,1.0,0.0,0.0
175,2820669.0,165868.0,-47.0,14.0,4.0,0.0,0.0,0.0,1.0,0.0


In [42]:
dp.peek_df(APPLICATIONS)

20000 rows × 588 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [43]:
pos_cash_agg_funcs = {
    "MONTHS_BALANCE": ["max", "mean", "size"],
    "SK_DPD": ["max", "mean"],
    "SK_DPD_DEF": ["max", "mean"],
}

for cat_col in dp.footprints.transformers[-1].target_col_names:
    pos_cash_agg_funcs[cat_col] = ["mean"]
    
pos_cash_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=POS_CASH_BALANCES,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=pos_cash_agg_funcs,
    target_col_prefix="POS_CASH"
)

dp = dp.fit_transform(pos_cash_aggregator)

In [44]:
dp.peek_df(APPLICATIONS)

20000 rows × 598 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

# 5. Installments Payments

In [45]:
dp.peek_df(INSTALLMENTS_PAYMENTS)

783857 rows × 8 columns


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470
10,1782554,106597,1.0,3,-933.0,-937.0,34620.525,34620.525
14,1594684,100193,0.0,56,-1153.0,-1153.0,301.860,301.860
21,2260765,106012,3.0,23,-56.0,-59.0,13494.915,13494.915


In [46]:
bicolumns_configs = [
    ("PAYMENT_PERC", "AMT_PAYMENT", "/","AMT_INSTALMENT"),
    ("PAYMENT_DIFF", "AMT_INSTALMENT", "-", "AMT_PAYMENT"),
    ("DPD", "DAYS_ENTRY_PAYMENT", get_dpd, "DAYS_INSTALMENT"),
    ("DBD", "DAYS_INSTALMENT", get_dbd, "DAYS_ENTRY_PAYMENT"),
]
bicolumns_pipeline = []
for target_col_name, first_col_name, operation, second_col_name in bicolumns_configs:
    bicolumns_pipeline.append(
        BicolumnsOperator(
            target_df_name=INSTALLMENTS_PAYMENTS,
            target_col_name=target_col_name,
            first_col_name=first_col_name,
            operation=operation,
            second_col_name=second_col_name,
        )
    )
bicolumns_pipeline = Serializer(transformers=bicolumns_pipeline)
    
dp = dp.fit_transform(bicolumns_pipeline)

In [47]:
dp.peek_df(INSTALLMENTS_PAYMENTS)

783857 rows × 12 columns


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,PAYMENT_PERC,PAYMENT_DIFF,DPD,DBD
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165,3.190569e+08,0.0,0.0,26.0
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470,2.047323e+08,0.0,0.0,39.0
10,1782554,106597,1.0,3,-933.0,-937.0,34620.525,34620.525,1.198581e+09,0.0,0.0,4.0
14,1594684,100193,0.0,56,-1153.0,-1153.0,301.860,301.860,9.111946e+04,0.0,0.0,0.0
21,2260765,106012,3.0,23,-56.0,-59.0,13494.915,13494.915,1.821127e+08,0.0,0.0,3.0


In [48]:
dp.peek_main_df()

20000 rows × 598 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [49]:
int_pay_agg_funcs = {
    "NUM_INSTALMENT_VERSION": ["nunique", "size"],
    "DPD": ["max", "mean", "sum"],
    "DBD": ["max", "mean", "sum"],
    "PAYMENT_PERC": ["max", "mean", "sum", "var"],
    "PAYMENT_DIFF": ["max", "mean", "sum", "var"],
    "AMT_INSTALMENT": ["max", "mean", "sum"],
    "AMT_PAYMENT": ["min", "max", "mean", "sum"],
    "DAYS_ENTRY_PAYMENT": ["max", "mean", "sum"],
}

int_pay_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=INSTALLMENTS_PAYMENTS,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=int_pay_agg_funcs,
    target_col_prefix="INT_PAY",
)

dp = dp.fit_transform(int_pay_aggregator)

In [50]:
dp.peek_main_df()

20000 rows × 624 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

# 6. CC Balances

In [51]:
dp.peek_df(CC_BALANCES)

227988 rows × 23 columns


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
6,1079071,171320,-6,353451.645,585000,67500.0,67500.0,0.0,0.0,14684.175,15750.00,15750.0,345433.860,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.0,0.0,0.0,0.000,264.69,0.0,44735.310,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
23,2021411,105755,-5,464452.470,450000,135000.0,279112.5,137700.0,6412.5,8921.970,9000.00,9000.0,448552.125,456379.470,456379.470,5.0,7,1.0,1.0,25.0,Active,0,0
46,1426111,116448,-7,0.000,0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.000,0.000,0.0,0,0.0,0.0,15.0,Completed,0,0
47,1517613,121258,-6,0.000,135000,NaN,0.0,NaN,NaN,0.000,NaN,0.0,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


In [52]:
df = dp.dfs[CC_BALANCES]
one_hot_encoder = OneHotEncoder(
    target_df_name=CC_BALANCES, col_names=get_categorical_column_names(df), fit_on_split=None
)
dp = dp.fit_transform(one_hot_encoder)

In [53]:
dp.peek_df(CC_BALANCES)

441974 rows × 24 columns


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_ACTIVE,NAME_CONTRACT_STATUS_INFREQUENT_SKLEARN
6,1079071.0,171320.0,-6.0,353451.645,585000.0,67500.0,67500.0,0.0,0.0,14684.175,15750.00,15750.0,345433.860,351881.145,351881.145,1.0,1.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0
7,2095912.0,118650.0,-7.0,47962.125,45000.0,45000.0,45000.0,0.0,0.0,0.000,264.69,0.0,44735.310,47962.125,47962.125,1.0,1.0,0.0,0.0,51.0,0.0,0.0,1.0,0.0
23,2021411.0,105755.0,-5.0,464452.470,450000.0,135000.0,279112.5,137700.0,6412.5,8921.970,9000.00,9000.0,448552.125,456379.470,456379.470,5.0,7.0,1.0,1.0,25.0,0.0,0.0,1.0,0.0
46,1426111.0,116448.0,-7.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,15.0,0.0,0.0,1.0,0.0
47,1517613.0,121258.0,-6.0,0.000,135000.0,NaN,0.0,NaN,NaN,0.000,NaN,0.0,0.000,0.000,0.000,NaN,0.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0


In [54]:
dp.peek_main_df()

20000 rows × 624 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

In [55]:
df = dp.dfs[CC_BALANCES]
col_names = list(df.columns)
col_names = [col_name for col_name in col_names if col_name not in {"SK_ID_PREV", "SK_ID_CURR"}]

cc_bal_funcs = {col_name: ['min', 'max', 'mean', 'sum', 'var'] for col_name in col_names}
cc_bal_funcs["MONTHS_BALANCE"].append("size")

cc_bal_aggregator = Aggregator(
    target_df_name=APPLICATIONS,
    source_df_name=CC_BALANCES,
    groupby_col_names=PRIMARY_KEY,
    agg_functions=cc_bal_funcs,
    target_col_prefix="CC_BALANCES"
)

dp = dp.fit_transform(cc_bal_aggregator)

In [56]:
dp.peek_main_df()

20000 rows × 735 columns


,SK_ID_CURR,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SPLIT,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE,NAME_CONTRACT_TYPE_CASH_LOANS,NAME_CONTRACT_TYPE_REVOLVING_LOANS,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_FAMILY,NAME_TYPE_SUITE_UNACCOMPANIED,NAME_TYPE_SUITE_INFREQUENT_SKLEARN,NAME_INCOME_TYPE_COMMERCIAL_ASSOCIATE,NAME_INCOME_TYPE_PENSIONER,NAME_INCOME_TYPE_STATE_SERVANT,NAME_INCOME_TYPE_WORKING,NAME_INCOME_TYPE_INFREQUENT_SKLEARN,NAME_EDUCATION_TYPE_HIGHER_EDUCATION,NAME_EDUCATION_TYPE_SECONDARY_SECONDARY_SPECIAL,NAME_EDUCATION_TYPE_INFREQUENT_SKLEARN,NAME_FAMILY_STATUS_CIVIL_MARRIAGE,NAME_FAMILY_STATUS_MARRIED,NAME_FAMILY_STATUS_SEPARATED,NAME_FAMILY_STATUS_SINGLE_NOT_MARRIED,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_HOUSE_APARTMENT,NAME_HOUSING_TYPE_INFREQUENT_SKLEARN,OCCUPATION_TYPE_CORE_STAFF,OCCUPATION_TYPE_DRIVERS,OCCUPATION_TYPE_LABORERS,OCCUPATION_TYPE_MANAGERS,OCCUPATION_TYPE_SALES_STAFF,OCCUPATION_TYPE_NAN,OCCUPATION_TYPE_INFREQUENT_SKLEARN,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3,ORGANIZATION_TYPE_OTHER,ORGANIZATION_TYPE_SELF_EMPLOYED,ORGANIZATION_TYPE_XNA,ORGANIZATION_TYPE_INFREQUENT_SKLEARN,FONDKAPREMONT_MODE_REG_OPER_ACCOUNT,FONDKAPREMONT_MODE_NAN,FONDKAPREMONT_MODE_INFREQUENT_SKLEARN,HOUSETYPE_MODE_BLOCK_OF_FLATS,HOUSETYPE_MODE_NAN,HOUSETYPE_MODE_INFREQUENT_SKLEARN,WALLSMATERIAL_MODE_PANEL,WALLSMATERIAL_MODE_STONE_BRICK,WALLSMATERIAL_MODE_NAN,WALLSMATERIAL_MODE_INFREQUENT_SKLEARN,EMERGENCYSTATE_MODE_NO,EMERGENCYSTATE_MODE_NAN,EMERGENCYSTATE_MODE_INFREQUENT_SKLEARN,BUREAUS_DAYS_CREDIT_MIN,BUREAUS_DAYS_CREDIT_MAX,BUREAUS_DAYS_CREDIT_MEAN,BUREAUS_DAYS_CREDIT_VAR,BUREAUS_DAYS_CREDIT_ENDDATE_MIN,BUREAUS_DAYS_CREDIT_ENDDATE_MAX,BUREAUS_DAYS_CREDIT_ENDDATE_MEAN,BUREAUS_DAYS_CREDIT_UPDATE_MEAN,BUREAUS_CREDIT_DAY_OVERDUE_MAX,BUREAUS_CREDIT_DAY_OVERDUE_MEAN,BUREAUS_AMT_CR

# Delete tables

In [57]:
dp.delete_dfs(
    [APPLICATIONS],
    is_delete_all_except_specified_df_names=True,
)

In [58]:
with open(cfg.research_cache_path / "03_dp.pkl", "wb") as f:
    pickle.dump(dp, f)